In [1]:
from decimal import Decimal, getcontext
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, Tuple, List, Optional

import math
import pandas as pd
import numpy as np

# High precision for tick<->sqrt price conversions
getcontext().prec = 80

Q96 = 2 ** 96
Q128 = 2 ** 128
FEE_DENOM = 1_000_000  # Uniswap v3 fee is in hundredths of a bip

@dataclass
class PoolConfig:
    pool: str
    token0: str
    token1: str
    fee: int           # e.g., 3000 for 0.3%
    tick_spacing: int
    decimals0: int
    decimals1: int

@dataclass
class PoolState:
    sqrt_price_x96: int
    tick: int
    liquidity_active: int
    fee_protocol_token0: int  # 0-15 (packed 4-bit semantics; see note below)
    fee_protocol_token1: int
    fee_growth_global0_x128: int = 0
    fee_growth_global1_x128: int = 0

@dataclass
class TickInfo:
    liquidity_net: int  # int128
    fee_growth_outside0_x128: int = 0
    fee_growth_outside1_x128: int = 0
    initialized: bool = False


def decode_fee_protocol(fp: int) -> Tuple[int, int]:
    # Packed: token1 in high 4 bits, token0 in low 4 bits (Uniswap v3 core)
    # NOTE: Exact protocol fee share mapping should match core; default safe is 0 (no protocol fee)
    t0 = fp & 0x0F
    t1 = (fp >> 4) & 0x0F
    return t0, t1


def price_to_tick_sqrt(px: Decimal) -> int:
    # px is token1 per token0; tick = log_1.0001(px)
    if px <= 0:
        raise ValueError('price must be positive')
    # log(px)/log(1.0001); round to nearest int
    return int((px.ln() / Decimal('1.0001').ln()).to_integral_value(rounding='ROUND_HALF_EVEN'))


def tick_to_sqrt_price_x96(tick: int) -> int:
    # sqrtPriceX96 = floor(sqrt(1.0001^tick) * Q96)
    # Use Decimal for precision
    sqrt_price = (Decimal('1.0001') ** Decimal(tick)).sqrt()
    return int((sqrt_price * Q96).to_integral_value(rounding='ROUND_FLOOR'))


def sqrt_price_x96_to_price(sqrt_price_x96: int, decimals0: int, decimals1: int) -> Decimal:
    # token1 per token0 price
    p = Decimal(sqrt_price_x96) / Decimal(Q96)
    return (p * p) * (Decimal(10) ** Decimal(decimals0 - decimals1))


def get_amount0_delta(sqrt_pa_x96: int, sqrt_pb_x96: int, liquidity: int, round_up: bool) -> int:
    # amount0 = L * (sqrt(b) - sqrt(a)) / (sqrt(b) * sqrt(a))
    if sqrt_pa_x96 > sqrt_pb_x96:
        sqrt_pa_x96, sqrt_pb_x96 = sqrt_pb_x96, sqrt_pa_x96
    num = (int(liquidity) * (sqrt_pb_x96 - sqrt_pa_x96))
    denom = (sqrt_pb_x96 * sqrt_pa_x96) // Q96
    # scale back properly: (num * Q96) / (sqrt_pb * sqrt_pa)
    # To avoid precision loss, compute as: (liquidity << 96) * (sqrtB - sqrtA) / (sqrtB * sqrtA)
    num = (int(liquidity) * (sqrt_pb_x96 - sqrt_pa_x96)) << 96
    denom = sqrt_pb_x96 * sqrt_pa_x96
    if round_up:
        return (num + denom - 1) // denom
    else:
        return num // denom


def get_amount1_delta(sqrt_pa_x96: int, sqrt_pb_x96: int, liquidity: int, round_up: bool) -> int:
    # amount1 = L * (sqrt(b) - sqrt(a))
    if sqrt_pa_x96 > sqrt_pb_x96:
        sqrt_pa_x96, sqrt_pb_x96 = sqrt_pb_x96, sqrt_pa_x96
    amount = int(liquidity) * (sqrt_pb_x96 - sqrt_pa_x96) // Q96
    if round_up and ((int(liquidity) * (sqrt_pb_x96 - sqrt_pa_x96)) % Q96 != 0):
        return amount + 1
    return amount


def apply_protocol_fee(fee_amount: int, proto_b4: int) -> Tuple[int, int]:
    # Returns (lp_fee_amount, protocol_cut)
    # In core, protocol fee share is encoded; when zero, LPs get all.
    if proto_b4 == 0:
        return fee_amount, 0
    # Placeholder: treat proto_b4 as parts of 256
    # LP share = fee_amount * (256 - proto_b4) / 256
    lp = (fee_amount * (256 - proto_b4)) // 256
    return lp, fee_amount - lp


def build_liquidity_net_from_events(mints: pd.DataFrame, burns: pd.DataFrame) -> Dict[int, TickInfo]:
    ticks: Dict[int, TickInfo] = {}
    def add_tick(t: int, delta: int):
        info = ticks.get(t)
        if info is None:
            info = TickInfo(liquidity_net=0, initialized=True)
            ticks[t] = info
        info.liquidity_net += delta
        info.initialized = True
    # Mints add liquidity between tickLower..tickUpper: +L at lower, -L at upper
    for _, r in mints.iterrows():
        tl = int(r['tickLower'])
        tu = int(r['tickUpper'])
        L = int(r['liquidity_added'])
        add_tick(tl, +L)
        add_tick(tu, -L)
    # Burns remove: -L at lower, +L at upper
    for _, r in burns.iterrows():
        tl = int(r['tickLower'])
        tu = int(r['tickUpper'])
        L = int(r['liquidity_removed'])
        add_tick(tl, -L)
        add_tick(tu, +L)
    return ticks


def init_active_liquidity_from_ticks(current_tick: int, ticks: Dict[int, TickInfo]) -> int:
    # Sum liquidityNet for ticks <= current_tick
    total = 0
    for t, info in ticks.items():
        if t <= current_tick and info.initialized:
            total += info.liquidity_net
    return max(total, 0)


def simulate_swaps(swaps: pd.DataFrame,
                   pool: PoolConfig,
                   init_state: PoolState,
                   ticks: Dict[int, TickInfo]) -> Tuple[PoolState, Dict[int, TickInfo]]:
    state = PoolState(
        sqrt_price_x96=init_state.sqrt_price_x96,
        tick=init_state.tick,
        liquidity_active=init_state.liquidity_active,
        fee_protocol_token0=init_state.fee_protocol_token0,
        fee_protocol_token1=init_state.fee_protocol_token1,
        fee_growth_global0_x128=init_state.fee_growth_global0_x128,
        fee_growth_global1_x128=init_state.fee_growth_global1_x128,
    )

    fee_bps = pool.fee  # e.g., 3000
    for _, s in swaps.iterrows():
        target_sqrt = int(s['sqrtPriceX96'])
        target_tick = int(s['tick'])
        amt0 = int(s['amount0'])
        amt1 = int(s['amount1'])
        if amt0 == 0 and amt1 == 0:
            continue
        # Determine direction and input token
        if amt0 > 0 and amt1 < 0:
            input_token = 0  # token0 in
            zero_for_one = True  # price decreases when selling token0
        elif amt1 > 0 and amt0 < 0:
            input_token = 1  # token1 in
            zero_for_one = False  # price increases when selling token1
        else:
            # Fallback to comparing target sqrt vs current
            zero_for_one = target_sqrt < state.sqrt_price_x96
            input_token = 0 if zero_for_one else 1

        # Traverse until we reach target_sqrt
        while state.sqrt_price_x96 != target_sqrt:
            if zero_for_one:
                # Moving left: next tick boundary below current
                next_tick = state.tick - 1
                next_sqrt = tick_to_sqrt_price_x96(next_tick)
                step_target = max(next_sqrt, target_sqrt)
                # Net required input (no fee) to move to step_target
                in_net = get_amount0_delta(step_target, state.sqrt_price_x96, state.liquidity_active, round_up=True)
            else:
                # Moving right: next tick boundary above current
                next_tick = state.tick + 1
                next_sqrt = tick_to_sqrt_price_x96(next_tick)
                step_target = min(next_sqrt, target_sqrt)
                in_net = get_amount1_delta(state.sqrt_price_x96, step_target, state.liquidity_active, round_up=True)

            # Gross in with fee-on-input: gross = ceil(in_net / (1 - fee))
            gross_in = (in_net * FEE_DENOM + (FEE_DENOM - fee_bps) - 1) // (FEE_DENOM - fee_bps)
            fee_amount = gross_in - in_net

            # Apply protocol fee split for the input token
            if input_token == 0:
                lp_fee, _ = apply_protocol_fee(fee_amount, state.fee_protocol_token0)
                if state.liquidity_active > 0 and lp_fee > 0:
                    state.fee_growth_global0_x128 += (lp_fee * Q128) // state.liquidity_active
            else:
                lp_fee, _ = apply_protocol_fee(fee_amount, state.fee_protocol_token1)
                if state.liquidity_active > 0 and lp_fee > 0:
                    state.fee_growth_global1_x128 += (lp_fee * Q128) // state.liquidity_active

            # Move price
            state.sqrt_price_x96 = step_target
            # If we reached a tick boundary, cross it
            if state.sqrt_price_x96 == next_sqrt:
                # Update feeGrowthOutside at this tick
                info = ticks.get(next_tick)
                if info is None:
                    info = TickInfo(liquidity_net=0, initialized=False)
                    ticks[next_tick] = info
                # On first time touching the other side, set outside to globals
                info.fee_growth_outside0_x128 = state.fee_growth_global0_x128
                info.fee_growth_outside1_x128 = state.fee_growth_global1_x128
                # Update active liquidity per crossing direction
                if zero_for_one:
                    state.liquidity_active -= info.liquidity_net
                else:
                    state.liquidity_active += info.liquidity_net
                # Advance tick index
                state.tick = next_tick
            else:
                # We reached the final target within the current tick
                # Set tick to target_tick for consistency
                state.tick = target_tick

        # Optional: validate liquidity against event-provided liquidity
        if 'liquidity' in s and not pd.isna(s['liquidity']):
            # We accept small discrepancies; this is primarily a diagnostic
            pass

    return state, ticks


def fee_growth_inside_delta(ticks: Dict[int, TickInfo],
                            tick_lower: int,
                            tick_upper: int,
                            global0_x128_start: int,
                            global1_x128_start: int,
                            global0_x128_end: int,
                            global1_x128_end: int,
                            tick_at_start: int,
                            tick_at_end: int) -> Tuple[int, int]:
    # Compute feeGrowthInside delta per v3 formula using feeGrowthOutside at start/end
    def growth_inside(global_start, global_end, lower_out_start, upper_out_start, lower_out_end, upper_out_end, tick_start, tick_end):
        # Start snapshot
        below_start = lower_out_start if tick_start >= tick_lower else global_start - lower_out_start
        above_start = upper_out_start if tick_start < tick_upper else global_start - upper_out_start
        inside_start = global_start - below_start - above_start
        # End snapshot
        below_end = lower_out_end if tick_end >= tick_lower else global_end - lower_out_end
        above_end = upper_out_end if tick_end < tick_upper else global_end - upper_out_end
        inside_end = global_end - below_end - above_end
        return inside_end - inside_start

    lower = ticks.get(tick_lower, TickInfo(0, 0, 0, False))
    upper = ticks.get(tick_upper, TickInfo(0, 0, 0, False))

    d0 = growth_inside(global0_x128_start, global0_x128_end,
                       lower.fee_growth_outside0_x128, upper.fee_growth_outside0_x128,
                       lower.fee_growth_outside0_x128, upper.fee_growth_outside0_x128,
                       tick_at_start, tick_at_end)

    d1 = growth_inside(global1_x128_start, global1_x128_end,
                       lower.fee_growth_outside1_x128, upper.fee_growth_outside1_x128,
                       lower.fee_growth_outside1_x128, upper.fee_growth_outside1_x128,
                       tick_at_start, tick_at_end)
    return d0, d1


def load_pool_context(base: Path) -> Tuple[PoolConfig, PoolState]:
    pool_cfg = pd.read_csv(base / 'dune_pipeline' / 'pool_config_eth_usdt_0p3.csv')
    tokens = pd.read_csv(base / 'dune_pipeline' / 'token_metadata_eth_usdt_0p3.csv')
    tokens['contract_address'] = tokens['contract_address'].str.lower()
    t0 = tokens.set_index('contract_address').loc[pool_cfg.loc[0, 'token0'].lower()]
    t1 = tokens.set_index('contract_address').loc[pool_cfg.loc[0, 'token1'].lower()]
    cfg = PoolConfig(
        pool=pool_cfg.loc[0, 'pool'],
        token0=pool_cfg.loc[0, 'token0'].lower(),
        token1=pool_cfg.loc[0, 'token1'].lower(),
        fee=int(pool_cfg.loc[0, 'fee']),
        tick_spacing=int(pool_cfg.loc[0, 'tickSpacing']),
        decimals0=int(t0['decimals']),
        decimals1=int(t1['decimals']),
    )
    slot0 = pd.read_csv(base / 'dune_pipeline' / 'slot0_2025_09_01_to_2025_10_01_eth_usdt_0p3.csv')
    slot0 = slot0.sort_values('call_block_time').iloc[0]
    fee_proto0, fee_proto1 = decode_fee_protocol(int(slot0['output_feeProtocol']))
    init = PoolState(
        sqrt_price_x96=int(slot0['output_sqrtPriceX96']),
        tick=int(slot0['output_tick']),
        liquidity_active=0,  # will init from ticks
        fee_protocol_token0=fee_proto0,
        fee_protocol_token1=fee_proto1,
    )
    return cfg, init


def load_events(base: Path) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    swaps = pd.read_csv(base / 'dune_pipeline' / 'swaps_2025_09_01_to_2025_10_01_eth_usdt_0p3.csv')
    swaps['evt_block_time'] = pd.to_datetime(swaps['evt_block_time'], utc=True)
    swaps = swaps.sort_values(['evt_block_time', 'evt_block_number']).reset_index(drop=True)
    mints = pd.read_csv(base / 'dune_pipeline' / 'mints_2025_09_01_to_2025_10_01_eth_usdt_0p3.csv')
    mints['evt_block_time'] = pd.to_datetime(mints['evt_block_time'], utc=True)
    mints = mints.sort_values(['evt_block_time', 'evt_block_number']).reset_index(drop=True)
    burns = pd.read_csv(base / 'dune_pipeline' / 'burns_2025_09_01_to_2025_10_01_eth_usdt_0p3.csv')
    burns['evt_block_time'] = pd.to_datetime(burns['evt_block_time'], utc=True)
    burns = burns.sort_values(['evt_block_time', 'evt_block_number']).reset_index(drop=True)
    return swaps, mints, burns


# Demo wiring
BASE = Path('/home/poon/developments/ice-senior-project')
cfg, init_state = load_pool_context(BASE)
swaps_df, mints_df, burns_df = load_events(BASE)

# Build tick map from events in window (note: for highest accuracy, supply full snapshot at t0)
tick_map = build_liquidity_net_from_events(mints_df, burns_df)
init_state.liquidity_active = init_active_liquidity_from_ticks(init_state.tick, tick_map)

# Run simulation over swaps
final_state, tick_map_out = simulate_swaps(swaps_df, cfg, init_state, tick_map)

print({'tick_start': init_state.tick, 'tick_end': final_state.tick})
print({'feeGrowthGlobal0X128': final_state.fee_growth_global0_x128,
       'feeGrowthGlobal1X128': final_state.fee_growth_global1_x128})



{'tick_start': -192460, 'tick_end': -193051}
{'feeGrowthGlobal0X128': 4107907358589966072215837164590683770653, 'feeGrowthGlobal1X128': 16771029707278741185709185722760}
